# **Agregação Bootstrap**

## Introdução

Neste tutorial aprenderemos como implementar o procedimento Agregação Bootstrap que é um técnica para fazer árvores de decisão mais robustas e alcançar melhor performace. 

Antes de vermos os algoritmos, vamos aprender alguns conceitos.

### Algoritmo de Agregação Bootstrap

Um bootstrap é uma amostra de um conjunto de dados com substituição. Isso significa que um novo conjunto é criado de uma amostra aletatória de um conjuntos de dados existente onde uma linha pode ser selecionada e adicionada mais do que
uma vez à amostra. Criando amostras de um conjunto de dados e estimando o significado dessas amostras podemos calcular
a média dessas estimativas e ter uma melhor idéia do verdadeiro significado do problema. <br />
Essa abordagem pode ser usada com algoritmos de aprendizado de máquina que têm alta variância. Um modelo separado é treinado sobre cada bootstrap e a média dos outputs são usados para fazer previsões. <br />
Variância significa que a performance é sensível aos dados de treino, com alta variação, sugerimos que quanto mais os dados de treinamento forem alterados, mais o desempenho do algoritmo variará. 
A performance de algoritmos de machine learning com alta variância pode ser melhorada treinando muitas árvores e 
calculando a média de suas previsões. Os resultados são geralmente melhores do que uma única árvore de decisão. Outro benefício do algoritmo, além do desempenho melhorado, é que as árvores de decisão geradas não podem superar o problema. As árvores podem continuar a ser adicionadas até um máximo em desempenho ser alcançado. 

<br />
<br />
Neste tutorial usaremos o conjunto de dados Sonar. Esse conjunto de dados é sobre a diferença entre minas e rochas. A performance desse exemplo é de aproximadamente 53%. Baixe o arquivo com os dados através do link https://github.com/DayanaRochaM/TutorialMachineLearning-/blob/master/sonar.all-data.csv e guarde-o no seu diretório de trabalho atual sem alterar seu nome (sonar.all-data.csv).

## Passos do tutorial
1. Bootstrap Resample
2. Caso de estudo Sonar

### 1. Bootstrap Resample

Vamos começar tendo uma ideia de como o método Bootstrap funciona. Podemos criar uma simples amostra de um conjunto 
    de dados selecionando linhas aleatórias e adicionando-as a uma nova lista. Abaixo está uma função chamada **subsample()** que implementa esse procedimento. <br /> 
<br />
A função auxiliar **randrange()** do módulo *random* é usada para
selecionar o índice de uma linha aleatória para ser adicionada à amostra em cada interação do loop. O valor padrão de
uma amostra é o tamanho do conjunto de dados original.
<br />
<br />

In [4]:
def subsample(dataset, ratio=1.0):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample


Podemos usar essa função para estimar a média de um conjunto de dados inventado, como é o caso do exemplo abaixo. Neste, primeiro criamos um conjunto de dados de 20 linhas e apenas uma coluna de números aleatórios entre 0 e 9,
calculamos a média e as amostras bootstrap são criadas com 10% do conjunto de dados original. Depois, testamos nosso código criando 1, 10, 100 amostras de bootstrap do conjunto de dados original, calculamos suas médias e então, média de todos os valores médios estimados. <br />
<br />
Abaixo usamos a função **seed()** para inicializar o estado interno do gerador de números aleatórios com "1".

In [8]:
from random import seed
from random import random
from random import randrange

# Cria uma subamostra aleatória do conjunto de dados com substituição
def subsample(dataset, ratio=1.0):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

# Calcula a média de uma lista de números
def mean(numbers):
    return sum(numbers) / float(len(numbers))

# Testa a subamostragem de um conjunto de dados --------
seed(1)

# Verdadeira média
dataset = [[randrange(10)] for i in range(20)]
print('True Mean: %.3f' % mean([row[0] for row in dataset])) 

# Média estimada
ratio = 0.10
for size in [1, 10, 100]:
    sample_means = list()
    for i in range(size):
        sample = subsample(dataset, ratio)
        sample_mean = mean([row[0] for row in sample])
        sample_means.append(sample_mean)
    print('Samples=%d, Estimated Mean: %.3f' % (size, mean(sample_means)))

True Mean: 4.500
Samples=1, Estimated Mean: 4.000
Samples=10, Estimated Mean: 4.700
Samples=100, Estimated Mean: 4.570


Executando o exemplo acima podemos observar que a melhor estimativa vem da amostragem com 100 dados. 

O nosso próximo passo é combinar as previsões de múltiplos modelos de bootstrap.

<br />
### 2. Estudo do caso Sonar

Agora vamos aplicar o nosso algoritmo no conjunto de dados Sonar. Assumiremos que o arquivo **sonar.all-data.csv** já
se encontra no seu diretório.
<br />
<br />
O algoritmo carregará o conjunto de dados, converterá os valores string para numéricos e a coluna de output será 
convertida para strings entre 0 e 1. Esses primeiros passos serão feitos com as funções auxiliares **load_csv()**, **str_column_to_float()** e **str_colimn_to_int**.

Usaremos validação cruzada entre *k* partes para estimar a performance do modelo de aprendizado sobre dados aleatórios. Isso significa que iremos avaliar e contruir *k* modelos e estimar a perfomance pela média do erro do
modelo. Precisão da classificação será usada para avaliar cada modelo. Para essa parte usaremos as funções 
**cross_validation_split()**, **accuracy_metric()** e **evaluate_algorithm()**.

Nós usaremos também a implementação de Classification and Regression Trees(CART) adaptado para à Agregação Bootstrap com as funções **teste_split()**, **gini_index()**, **get_split()**, **to_terminal()**, **split()**, **build_tree()**,
**predict()**, e **subsample()**. Por fim, usaremos **bagging_predict()**, **bagging()**. <br />
<p style="color:red">O papel de cada função será apresentado como comentário abaixo, com a implementação do Algoritmo
de Agregação Bootstrap aplicado ao conjunto de dados Sonar.</p>

In [ ]:
# Algoritmo de Agregação Bootstrap sobre o conjunto de dados Sonar
from random import seed
from random import randrange
from csv import reader

# Carregar arquivo CSV
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

# Converter coluna string para float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())
        
# Converter coluna string para inteiro
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Dividir um conjunto de dados em k partes
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Calcular porcentagem de aproximação
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Avaliar um algoritmo usando uma divisão de validação cruzada
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

# Dividir um conjunto de dados com base em um atributo e um valor de atributo
def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calcular o índice Gini para um conjunto de dados dividido
def gini_index(groups, classes):
    # contar todas as amostras no ponto de divisão
    n_instances = float(sum([len(group) for group in groups]))
    # índice ponderado de Gini para cada grupo
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # evitar divisão por zero
        if size == 0:
            continue
        score = 0.0
        # marcar a pontuação do grupo na pontuação para cada classe
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # pontuação do grupo pelo tamanho relativo
        gini += (1.0 - score) * (size / n_instances)
    return gini

# Selecione o melhor ponto de divisão para um conjunto de dados
def get_split(dataset): 
    class_values = list(set(row[-1] for row in dataset)) 
    b_index, b_value, b_score, b_groups = 999, 999, 999, None 
    for index in range(len(dataset[0])-1): 
        for row in dataset: 
            # for i in range(len(dataset)):
            # row = dataset[randrange(len(dataset))]
            groups = test_split(index, row[index], dataset) 
            gini = gini_index(groups, class_values) 
            if gini < b_score: 
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups 
                return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Criar um valor de nó terminal
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

# Crie filhos para um nó ou faça um terminal
def split(node, max_depth, min_size, depth): 
    left, right = node['groups'] 
    del(node['groups'])
    # verifique se não há uma divisão
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right) 
        return
    # Verifique a profundidade máxima
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right) 
        return
    # processar filho esquerdo
    if len(left) <= min_size:
        node['left'] = to_terminal(left) 
    else: 
        node['left'] = get_split(left) 
        split(node['left'], max_depth, min_size, depth+1) 
    # processar filho direito 
    if len(right) <= min_size: 
        node['right'] = to_terminal(right) 
    else: 
        node['right'] = get_split(right) 
        split(node['right'], max_depth, min_size, depth+1)

# Construir uma árvore de decisão
def build_tree(train, max_depth, min_size): 
    root = get_split(train) 
    split(root, max_depth, min_size, 1) 
    return root
      
# Fazer uma previsão com a árvore de decisão
def predict(node, row): 
    if row[node['index']] < node['value']: 
        if isinstance(node['left'], dict): 
            return predict(node['left'], row) 
        else: 
            return node['left']
        else: 
            if isinstance(node['right'], dict): 
                return predict(node['right'], row) 
            else:
                return node['right']

            
# Crie uma subamostra aleatória do conjunto de dados com substituição 
def subsample(dataset, ratio): 
    sample = list() 
    n_sample = round(len(dataset) * ratio) 
    while len(sample) < n_sample: 
        index = randrange(len(dataset)) 
        sample.append(dataset[index]) 
    return sample

# Fazer uma previsão com uma lista de árvores bootstrap
def bagging_predict(trees, row): 
    predictions = [predict(tree, row) for tree in trees] 
    return max(set(predictions), key=predictions.count)

# Algoritmo de Agregação Bootstrap
def bagging(train, test, max_depth, min_size, sample_size, n_trees): 
    trees = list() 
    for i in range(n_trees): 
        sample = subsample(train, sample_size) 
        tree = build_tree(sample, max_depth, min_size) 
        trees.append(tree) 
    predictions = [bagging_predict(trees, row) for row in test] 
    return(predictions)

# Testando o algoritmo no Sonar 
dataset seed(1)

# Carregar e preparar dados
filename = 'sonar.all-data.csv' 
dataset = load_csv(filename) 
# converter atributos string para inteiros
for i in range(len(dataset[0])-1): 
    str_column_to_float(dataset, i) 
# converter a coluna da classe para números inteiros
str_column_to_int(dataset, len(dataset[0])-1) 

# avaliar algoritmo
n_folds = 5 
max_depth = 6 
min_size = 2 
sample_size = 0.50 
for n_trees in [1, 5, 10, 50]: 
    scores = evaluate_algorithm(dataset, bagging, n_folds, max_depth, min_size, sample_size, n_trees) 
    print('Trees: %d' % n_trees) 
    print('Scores: %s' % scores) 
    print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))
